# Import Libraries

In [1]:
# Import dataset utilities
import kagglehub
import os

#Import data structures
import pandas as pd
import numpy as np

# Custom Functions

## get_flight_status()

In [2]:
# Custom function used to create target label for classification problem
def get_flight_status(row):
    if row["Cancelled"] == 1:   # Determines if flight is canceled
        return "Canceled"

    if row["DepDel15"] > 0:     # Determines if delay in departure
        return "Delayed"
    else:
        return "On Time"        # Otherwise, flight is on time


# Prepare Dataset

## Retrieve Dataset from Kaggle

In [3]:
# Download the path to access dataset
path = kagglehub.dataset_download("robikscube/flight-delay-dataset-20182022")

# Verify the path
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flight-delay-dataset-20182022


In [4]:
# Relevant columns to keep
cols_keep = [
    'FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'CRSDepTime',
    'DepTime', 'DepDelay', 'ArrTime', 'CRSElapsedTime', 'Distance', 'Quarter',
    'Marketing_Airline_Network', 'Year', 'Month', 'DayOfWeek',
    'DOT_ID_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline',
    'Flight_Number_Marketing_Airline', 'OriginAirportID', 'OriginStateFips',
    'DestAirportID', 'DestStateFips', 'DepDel15', 'DepTimeBlk', 'CRSArrTime',
    'ArrDelay', 'ArrDel15', 'ArrTimeBlk', "Operating_Airline"
    ]

In [5]:
flights_2018 = pd.read_parquet(path + "/" + "Combined_Flights_2018.parquet", columns=cols_keep)  # Flight data from 2018
flights_2019 = pd.read_parquet(path + "/" + "Combined_Flights_2019.parquet", columns=cols_keep)  # Flight data from 2019
flights_2020 = pd.read_parquet(path + "/" + "Combined_Flights_2020.parquet", columns=cols_keep)  # Flight data from 2020
flights_2021 = pd.read_parquet(path + "/" + "Combined_Flights_2021.parquet", columns=cols_keep)  # Flight data from 2021
flights_2022 = pd.read_parquet(path + "/" + "Combined_Flights_2022.parquet", columns=cols_keep)  # Flight data from 2022

In [6]:
del cols_keep, path

## Combine Datasets

In [7]:
# Dictionary containing datasets for each year
datasets = {
    "2018": flights_2018,
    "2019": flights_2019,
    "2020": flights_2020,
    "2021": flights_2021,
    "2022": flights_2022
}

In [8]:
# Add a "Year" column to each dataset
for year, df in datasets.items():
    df["Year"] = int(year)  # Convert year to integer
    print(f"Added Year Column to {year}")

Added Year Column to 2018
Added Year Column to 2019
Added Year Column to 2020
Added Year Column to 2021
Added Year Column to 2022


In [9]:
# Merge the datasets and then delete them
all_flights = pd.concat(datasets.values(), ignore_index=True)

del datasets, flights_2018, flights_2019, flights_2020, flights_2021, flights_2022

## Clean Dataset

In [10]:
# Convert all columns to string
object_columns = all_flights.select_dtypes(include=['object']).columns  # Get list of applicable columns
all_flights[object_columns] = all_flights[object_columns].astype("string")  # and convert stored values to String

In [11]:
# Turn Cancelled to integer type indicating boolean values
all_flights["Cancelled"] = all_flights["Cancelled"].astype(int)

In [12]:
columns_to_fill_zero = [
    "DepTime", "DepDelay", "ArrTime", "ArrDelay", "DepDel15", "ArrDel15"
]

# Fill all the listed columns with 0 if the flight is cancelled
all_flights.loc[(all_flights["Cancelled"] == 1), columns_to_fill_zero] = 0

In [13]:
# Fill any missing string values to unknown
all_flights.loc[:, all_flights.select_dtypes(include=['string']).columns] = \
    all_flights.select_dtypes(include=['string']).fillna("Unknown")

In [14]:
# fill all numeric values to median statistic due to low percentage of missing values
cols_to_fill = all_flights.columns[all_flights.isnull().any()].tolist()

for col in cols_to_fill:
    all_flights[col].fillna(all_flights[col].median(), inplace=True)

<ipython-input-14-38a6b9aff60e>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_flights[col].fillna(all_flights[col].median(), inplace=True)


## Feature Selection & Engineering

Create target class, flight_status, which indicates whether flights are on time, delayed, or cancelled.

In [15]:
# Use get_flight_status() to create target class
all_flights["flight_status"] = all_flights.apply(get_flight_status, axis=1)

In [16]:
# Convert flight_status to String
all_flights["flight_status"] = all_flights["flight_status"].astype("string")

Feature engineer a previous flight delay to detect missed connections from short layovers

In [17]:
# Sort data for grouping logic
all_flights.sort_values(by=["FlightDate", "Tail_Number", "DepTime"], inplace=True)

In [18]:
# Shift values to get previous arrival and origin
all_flights["Prev_Arrtime"] = all_flights.groupby(['FlightDate', 'Tail_Number'])['ArrTime'].shift(1)
all_flights["Prev_Dest"]  = all_flights.groupby(['FlightDate', 'Tail_Number'])['Dest'].shift(1)
#

In [19]:
# Calculate layover time (departure - previous arrival)
all_flights["Layover_Time"] = all_flights["DepTime"] - all_flights["Prev_Arrtime"]

In [20]:
# Only keep layovers where previous destination matches current origin
all_flights.loc[all_flights["Prev_Dest"] != all_flights["Origin"], "Layover_Time"] = -1

In [21]:
# Mark flights with short layovers (< 45 minutes)
all_flights['ShortLayover'] = (all_flights['Layover_Time'] > 0) & (all_flights['Layover_Time'] < 45)

In [22]:
# Mark if previous flight was delayed (ArrDelay > 15)
all_flights['Prev_ArrDelay'] = all_flights.groupby(['FlightDate', 'Tail_Number'])['ArrDelay'].shift(1)
all_flights['PrevFlightDelayed'] = (all_flights['Prev_ArrDelay'] > 15).fillna(False)


In [23]:
# Combine both to create a final indicator
all_flights["Layover_Risk"] = all_flights["ShortLayover"] & all_flights["PrevFlightDelayed"]

In [24]:
# Review first five rows
all_flights.head()

,FlightDate,Airline,Origin,Dest,Cancelled,CRSDepTime,DepTime,DepDelay,ArrTime,CRSElapsedTime,...,ArrTimeBlk,Operating_Airline,flight_status,Prev_Arrtime,Prev_Dest,Layover_Time,ShortLayover,Prev_ArrDelay,PrevFlightDelayed,Layover_Risk
349626,2018-01-01,Allegiant Air,FLL,MEM,0,600,607.0,7.0,748.0,153.0,...,0700-0759,G4,On Time,NaN,<NA>,NaN,False,NaN,False,False
349627,2018-01-01,Allegiant Air,MEM,FLL,0,823,826.0,3.0,1144.0,136.0,...,1100-1159,G4,On Time,748.0,MEM,78.0,False,15.0,False,False
349628,2018-01-01,Allegiant Air,FLL,GSP,0,1229,1246.0,17.0,1451.0,112.0,...,1400-1459,G4,Delayed,1144.0,FLL,102.0,False,5.0,False,False
349629,2018-01-01,Allegiant Air,GSP,FLL,0,1506,1531.0,25.0,1732.0,109.0,...,1600-1659,G4,Delayed,1451.0,GSP,80.0,False,30.0,True,False
349630,2018-01-01,Allegiant Air,FLL,ORF,0,1745,1853.0,68.0,2058.0,125.0,...,1900-1959,G4,Delayed,1732.0,FLL,121.0,False,37.0,True,False


In [25]:
all_flights["Layover_Risk"].value_counts()

,count
Layover_Risk,
False,28564269
True,629513


In [26]:
all_flights["PrevFlightDelayed"].value_counts()

,count
PrevFlightDelayed,
False,25923219
True,3270563


In [27]:
all_flights["Prev_ArrDelay"].value_counts()

,count
Prev_ArrDelay,
0.0,1148724
-11.0,667621
-12.0,665068
-10.0,664939
-9.0,654506
...,...
1518.0,1
2677.0,1
4318.0,1


In [28]:
all_flights['ShortLayover'].value_counts()

,count
ShortLayover,
False,27177166
True,2016616


In [29]:
all_flights["Layover_Time"].value_counts()

,count
Layover_Time,
-1.0,694006
97.0,338543
98.0,337902
96.0,337389
99.0,337213
...,...
-450.0,1
-2338.0,1
-966.0,1


In [30]:
all_flights["Prev_Dest"].value_counts()

,count
Prev_Dest,
ATL,1169218
ORD,1165159
DEN,1009905
DFW,918287
CLT,816785
...,...
CDB,252
BFM,201
ILG,164


In [31]:
all_flights["Prev_Arrtime"].value_counts()

,count
Prev_Arrtime,
0.0,740063
1450.0,29856
1625.0,28949
1629.0,28904
1633.0,28816
...,...
2347.0,7
2354.0,7
2352.0,7


In [32]:
# Check if new attributes have NaN values
all_flights[["Prev_Arrtime", "Prev_Dest", "Layover_Time",
             'ShortLayover', "Prev_ArrDelay", "PrevFlightDelayed",
             "Layover_Risk"]].isnull().sum()

,0
Prev_Arrtime,7038927
Prev_Dest,7038927
Layover_Time,7038927
ShortLayover,0
Prev_ArrDelay,7038927
PrevFlightDelayed,0
Layover_Risk,0


In [33]:
# Null values mean no values, so fill float data types with 0.0
cols_to_fix = ["Prev_ArrDelay", "Layover_Time", "Prev_Arrtime"]
all_flights[cols_to_fix] = all_flights[cols_to_fix].fillna(0.0).astype(float)

# Fill String null values with None - No Previous Destination
all_flights["Prev_Dest"] = all_flights["Prev_Dest"].fillna("None")

In [34]:
all_flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29193782 entries, 349626 to 26470375
Data columns (total 39 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   FlightDate                       datetime64[us]
 1   Airline                          string        
 2   Origin                           string        
 3   Dest                             string        
 4   Cancelled                        int64         
 5   CRSDepTime                       int64         
 6   DepTime                          float64       
 7   DepDelay                         float64       
 8   ArrTime                          float64       
 9   CRSElapsedTime                   float64       
 10  Distance                         float64       
 11  Quarter                          int64         
 12  Marketing_Airline_Network        string        
 13  Year                             int64         
 14  Month                           

In [35]:
all_flights[["Prev_Arrtime", "Layover_Time", 'ShortLayover',
             "Prev_ArrDelay", "PrevFlightDelayed", "Layover_Risk"]].isnull().sum()

,0
Prev_Arrtime,0
Layover_Time,0
ShortLayover,0
Prev_ArrDelay,0
PrevFlightDelayed,0
Layover_Risk,0


Drop the columns risking data leakage

In [36]:
# Delete the columns that leak info
all_flights.drop(columns=["Cancelled",
                          "DepDel15",
                          "ArrDelay",
                          "DepTime",
                          "ArrDel15"],
                          inplace=True)

In [37]:
all_flights["flight_status"].value_counts()

,count
flight_status,
On Time,23512421
Delayed,4904094
Canceled,777267


In [38]:
# Print first five rows
all_flights.head()

,FlightDate,Airline,Origin,Dest,CRSDepTime,DepDelay,ArrTime,CRSElapsedTime,Distance,Quarter,...,ArrTimeBlk,Operating_Airline,flight_status,Prev_Arrtime,Prev_Dest,Layover_Time,ShortLayover,Prev_ArrDelay,PrevFlightDelayed,Layover_Risk
349626,2018-01-01,Allegiant Air,FLL,MEM,600,7.0,748.0,153.0,851.0,1,...,0700-0759,G4,On Time,0.0,None,0.0,False,0.0,False,False
349627,2018-01-01,Allegiant Air,MEM,FLL,823,3.0,1144.0,136.0,851.0,1,...,1100-1159,G4,On Time,748.0,MEM,78.0,False,15.0,False,False
349628,2018-01-01,Allegiant Air,FLL,GSP,1229,17.0,1451.0,112.0,620.0,1,...,1400-1459,G4,Delayed,1144.0,FLL,102.0,False,5.0,False,False
349629,2018-01-01,Allegiant Air,GSP,FLL,1506,25.0,1732.0,109.0,620.0,1,...,1600-1659,G4,Delayed,1451.0,GSP,80.0,False,30.0,True,False
349630,2018-01-01,Allegiant Air,FLL,ORF,1745,68.0,2058.0,125.0,781.0,1,...,1900-1959,G4,Delayed,1732.0,FLL,121.0,False,37.0,True,False


In [39]:
all_flights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29193782 entries, 349626 to 26470375
Data columns (total 34 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   FlightDate                       datetime64[us]
 1   Airline                          string        
 2   Origin                           string        
 3   Dest                             string        
 4   CRSDepTime                       int64         
 5   DepDelay                         float64       
 6   ArrTime                          float64       
 7   CRSElapsedTime                   float64       
 8   Distance                         float64       
 9   Quarter                          int64         
 10  Marketing_Airline_Network        string        
 11  Year                             int64         
 12  Month                            int64         
 13  DayOfWeek                        int64         
 14  DOT_ID_Operating_Airline        

In [40]:
#First way: Access your .ipynb file in google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [41]:
# Save the cleaned dataset as csv
all_flights.to_csv('/content/drive/MyDrive/CSCI334/Project/clean_flight_status.csv', index=False)


In [ ]:
!pip install nbconvert
# You also need to install latex-related package in order to convert a notebook to pdf file.
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc

In [ ]:
!apt-get update
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc -y


In [45]:
!cp /content/drive/MyDrive/CSCI334/Project/ FlightStatus_DataCleaning.ipynb ./
!jupyter nbconvert --to pdf /content/drive/MyDrive/CSCI334/Project/ FlightStatus_DataCleaning.ipynb

cp: -r not specified; omitting directory '/content/drive/MyDrive/CSCI334/Project/'
cp: cannot stat 'FlightStatus_DataCleaning.ipynb': No such file or directory
[NbConvertApp] WARNING | pattern 'FlightStatus_DataCleaning.ipynb' matched no files
[NbConvertApp] Converting notebook /content/drive/MyDrive/CSCI334/Project/ to pdf
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/jupyter_core/application.py", line 283, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 420, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.11/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_sin